In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random

In [ ]:
total_cars = 200
n_agents = 200 #fully-controlled

parked_agents_file = r"unisa_net/logs/unisa_parked_agents"
speed_meanagents_file = r"unisa_net/logs/unisa_speed_agents"
co2_agents_file = r"unisa_net/logs/unisa_co2_agents"
co_agents_file = r"unisa_net/logs/unisa_co_agents"
traveltime_agents_file = r"unisa_net/logs/unisa_traveltime_agents"
fuel_agents_file = r"unisa_net/logs/unisa_fuel_agents"
noise_agents_file= r"unisa_net/logs/unisa_noise_agents"
co2_meanagents_file = r"unisa_net/logs/unisa_co2_agentsmean"
co_meanagents_file = r"unisa_net/logs/unisa_co_agentsmean"
fuel_meanagents_file = r"unisa_net/logs/unisa_fuel_agentsmean"
noise_meanagents_file= r"unisa_net/logs/unisa_noise_agentsmean"

parked_foes_file = r"unisa_net/logs/unisa_parked_foes"
speed_meanfoes_file = r"unisa_net/logs/unisa_speed_foes"
co2_foes_file = r"unisa_net/logs/unisa_co2_foes"
co_foes_file = r"unisa_net/logs/unisa_co_foes"
traveltime_foes_file = r"unisa_net/logs/unisa_traveltime_foes"
fuel_foes_file = r"unisa_net/logs/unisa_fuel_foes"
noise_foes_file= r"unisa_net/logs/unisa_noise_foes"
co2_meanfoes_file = r"unisa_net/logs/unisa_co2_foesmean"
co_meanfoes_file = r"unisa_net/logs/unisa_co_foesmean"
fuel_meanfoes_file = r"unisa_net/logs/unisa_fuel_foesmean"
noise_meanfoes_file= r"unisa_net/logs/unisa_noise_foesmean"

traveltime_plot = r"unisa_net/logs/plots/unisa_traveltime_plot"
parked_cars_plot = r"unisa_net/logs/plots/unisa_parked_plot"
cars_co2_plot = r"unisa_net/logs/plots/unisa_co2_plot"
cars_co_plot = r"unisa_net/logs/plots/unisa_co_plot"
cars_fuel_plot = r"unisa_net/logs/plots/unisa_fuel_plot"
cars_noise_plot= r"unisa_net/logs/plots/unisa_noise_plot"
unparked_cars_plot = r"unisa_net/logs/plots/unisa_unparked_plot"

In [ ]:
n_simulations = 10
step_length = 0.05

In [ ]:
# The function get_steps provides the number of steps for each simulation.
def get_steps (file, offset):
    steps_simulations = [0]*n_simulations
    for i in range(n_simulations):
        arr = np.load(file + str(i+offset) + '.npy')
        steps_simulations[i] = arr.size
    return steps_simulations

In [ ]:
# The function getMeanSTD is used especially for the parked_cars file. Since simulations have different number of steps, each simulation  is enlarged to the maximum number of step by copying the last value.
# For each simulation, it computes the mean and stdev of parked cars.
def getMeanSTD(file, steps_simulations, offset):
    max_steps = max(steps_simulations)
    res = np.zeros((n_simulations,max_steps))
    for i in range(n_simulations):
        res[i, 0:steps_simulations[i]] = np.load(file + str(i+offset) + '.npy')
        res[i, steps_simulations[i] : max_steps] = res[i,steps_simulations[i]-1]
    means = [0]*max_steps
    stds = [0]*max_steps
    for j in range(max_steps):
        means[j] = np.mean(res[:, j])
        stds[j] = np.std(res[:, j])
    return(np.array(means), np.array(stds)), max_steps

In [ ]:
# The function getMeanSTD2 is used especially to compute the average emissions and fuel consumption for each step of the simulations.
# It computes the mean and stdev of values contained in file among all the simulations.
def getMeanSTD2(file, steps_simulations, offset): 
    max_steps = max(steps_simulations)
    res = np.zeros((n_simulations,max_steps))
    for i in range(n_simulations):
        res[i, 0:steps_simulations[i]] = np.load(file + str(i+offset) + '.npy')
    res*= step_length # to get the value from mg/s to mg/step
    means = [0]*max_steps
    stds = [0]*max_steps
    for j in range(max_steps):
        new_res = []
        for i in range(n_simulations):
            if (j < steps_simulations[i]): 
                new_res.append(res[i,j])
        means[j] = np.mean(new_res)
        stds[j] = np.std(new_res)
    return(np.array(means), np.array(stds)), max_steps

In [ ]:
# The function avgTravelTime returns the average time spent by a car during its journey to find the target parking area.
def avgTravelTime(file, offset):
    res = [0]*n_simulations
    for i in range(n_simulations):
        arr = np.load(file + str(i+offset) + '.npy')
        arr2 = []
        for xxx in arr:
            if xxx !=0:
                arr2.append(xxx)
        res[i] = np.mean(arr2)
    return(res)

In [ ]:
timeA= avgTravelTime(traveltime_agents_file,0)
timeF= avgTravelTime(traveltime_foes_file,0)
   
print(f"avg parking time CONTROLLED CARS: {np.mean(timeA)} - std {np.std(timeA)}")
print(f"avg parking time UNCONTROLLED CARS: {np.mean(timeF)} - std {np.std(timeF)}") 
X = np.arange(n_simulations)
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
ax.bar(X + 0.00, timeA, color = 'r', width = 0.25)
ax.bar(X + 0.25, timeF, color = 'b', width = 0.25)

ax.set_xlabel('Simulations')
ind = np.arange(n_simulations)    # the x locations for the groups
ax.set_xticks(ind)
ax.set_ylabel('Average Parking Time [s]')
ax.set_title('Average time spent by a car looking for the target parking area')
ax.legend(labels=['Controlled Cars (CC)', 'Uncontrolled Cars (UC)'])
fig = ax
fig.figure.savefig(traveltime_plot +'.pdf')
fig.figure.savefig(traveltime_plot +'.png', transparent=True)

plt.show()

In [ ]:
# Print the variance of both data groups
data_group1 = timeA[:(n_simulations)//2]
data_group2 = timeA[(n_simulations)//2:]
print(np.var(data_group1), np.var(data_group2))
#When the ratio is greater than 4: 1 hence the variance is different. So, we can apply Welch’s t-test (equal_var=False).

In [ ]:
#np_value
from scipy import stats  # https://www.geeksforgeeks.org/welchs-t-test-in-python/
stats.ttest_ind(timeA[:(n_simulations)//2], timeA[(n_simulations)//2:], equal_var=False) #result statistically significant

In [ ]:
#PLOT: Parked Controlled cars vs Parked Uncontrolled cars
steps_simulationsA = get_steps(parked_agents_file, 0)
steps_simulationsF = get_steps(parked_foes_file, 0)
        
res_agents, max_stepsA = getMeanSTD(parked_agents_file,steps_simulationsA, 0)
res_foes, max_stepsF = getMeanSTD(parked_foes_file, steps_simulationsF, 0)

endt = 40000
maxA = np.max(res_agents[0][0:endt])
x = np.array([t*step_length for t in range(max_stepsA)])
plt.plot(x[0:endt], res_agents[0][0:endt]*100/maxA, label = 'Mean parked CC', color='r')
upper = res_agents[0][0:endt]*100/maxA + res_agents[1][0:endt]*100/maxA
for xxx in range(len(upper)):
    if upper[xxx] >100:
        upper[xxx] =100     
        
plt.fill_between(x[0:endt], res_agents[0][0:endt]*100/maxA - res_agents[1][0:endt]*100/maxA, upper, color='r', alpha = 0.1,label = 'Std parked CC' )

x = np.array([t*step_length for t in range(max_stepsF)])
maxF = np.max(res_foes[0][0:endt])

plt.plot(x[0:endt], res_foes[0][0:endt]*100/total_cars, label = 'Mean parked UC', color='b') #total_cars -> fully uncontrolled 
upper = res_foes[0][0:endt]*100/total_cars + res_foes[1][0:endt]*100/total_cars
for xxx in range(len(upper)):
    if upper[xxx] >100:
        upper[xxx] =100  
plt.fill_between(x[0:endt], res_foes[0][0:endt]*100/total_cars - res_foes[1][0:endt]*100/total_cars, upper, color='b', alpha = 0.1, label = 'Std parked UC' )

# naming the x axis
plt.xlabel('time [s]')
# naming the y axis
plt.ylabel('Percentage of parked cars [%]')
 
plt.xticks(fontsize=14)
plt.rcParams.update({'font.size': 19})
 
# giving a title to my graph
plt.title('Parked cars over '+str(n_simulations)+' simulations')
# show a legend on the plot
plt.legend(fontsize=18)
plt.xlim([-50,2000])
plt.ylim([-1,101])
# resize the plot  
plt.rcParams["figure.figsize"] = (10, 10)
plt.grid()
# function to show the plot
plt.savefig(parked_cars_plot +'.pdf')
plt.savefig(parked_cars_plot +'.png', transparent=True)
 
plt.show()

In [ ]:
#PLOT: CO2 agents vs CO2 foes
steps_simulationsA = get_steps(co2_agents_file, 0)
steps_simulationsF = get_steps(co2_foes_file, 0)

res_agents, max_stepsA = getMeanSTD2(co2_agents_file,steps_simulationsA, 0)
res_foes, max_stepsF = getMeanSTD2(co2_foes_file, steps_simulationsF, 0)
#TO FILTER NOISY SIGNALS
from scipy.signal import lfilter
n = 80  # the larger n is, the smoother curve will be
b = [1.0 / n] * n
a = 1

res_agents[0][:] = lfilter(b,a,res_agents[0][:])
res_foes[0][:] = lfilter(b,a,res_foes[0][:])

#agents
endt = max_stepsA
x = np.array([t*step_length for t in range(max_stepsA)])
    
plt.plot(x[0:endt], res_agents[0][0:endt], label = 'Mean CO2 CC', color='r')
lower = res_agents[0][0:endt] - res_agents[1][0:endt]
for xxx in range(len(lower)):
    if lower[xxx] <0:
        lower[xxx] =0        
plt.fill_between(x[0:endt], lower, (res_agents[0][0:endt] + res_agents[1][0:endt]), color='r', alpha = 0.1, label = 'Std CO2 CC')

#foes
endt = max_stepsF
x = np.array([t*step_length for t in range(max_stepsF)])
plt.plot(x[0:endt], res_foes[0][0:endt], label = 'Mean CO2 UC', color='b')
lower = res_foes[0][0:endt] - res_foes[1][0:endt]
for xxx in range(len(lower)):
    if lower[xxx] <0:
        lower[xxx] =0        
plt.fill_between(x[0:endt], lower, (res_foes[0][0:endt] + res_foes[1][0:endt]), color='b', alpha = 0.1, label = 'Std CO2 UC')

# naming the x axis
plt.xlabel('time [s]')
# naming the y axis
plt.ylabel('Average CO2 [mg/s]')

plt.xticks(fontsize=14)
plt.rcParams.update({'font.size': 19})

# giving a title to my graph
plt.title('Average CO2 emissions over '+str(n_simulations)+' simulations')

# show a legend on the plot
plt.legend(fontsize=18)

# resize the plot  
plt.rcParams["figure.figsize"] = (10, 10)
plt.grid()
plt.xlim([-50,2000])
plt.ylim([-10,300])
# function to show the plot
plt.savefig(cars_co2_plot +'.pdf')
plt.savefig(cars_co2_plot +'.png', transparent=True)

plt.show()

In [ ]:
#PLOT: CO agents vs CO foes

steps_simulationsA = get_steps(co_agents_file, 0)
steps_simulationsF = get_steps(co_foes_file, 0)

res_agents, max_stepsA = getMeanSTD2(co_agents_file,steps_simulationsA, 0)
res_foes, max_stepsF = getMeanSTD2(co_foes_file, steps_simulationsF, 0)

#TO FILTER NOISY SIGNALS
from scipy.signal import lfilter
n = 80  # the larger n is, the smoother curve will be
b = [1.0 / n] * n
a = 1

res_agents[0][:] = lfilter(b,a,res_agents[0][:])
res_foes[0][:] = lfilter(b,a,res_foes[0][:])

#agents
endt = max_stepsA
x = np.array([t*step_length for t in range(max_stepsA)])

        
plt.plot(x[0:endt], res_agents[0][0:endt], label = 'Mean CO CC', color='r')
lower = res_agents[0][0:endt] - res_agents[1][0:endt]
for xxx in range(len(lower)):
    if lower[xxx] <0:
        lower[xxx] =0        
plt.fill_between(x[0:endt], lower, (res_agents[0][0:endt] + res_agents[1][0:endt]), color='r', alpha = 0.1, label = 'Std CO CC')

#foes
x = np.array([t*step_length for t in range(max_stepsF)])
    
plt.plot(x[0:endt], res_foes[0][0:endt], label = 'Mean CO UC', color='b')
lower = res_foes[0][0:endt] - res_foes[1][0:endt]
for xxx in range(len(lower)):
    if lower[xxx] <0:
        lower[xxx] =0        
plt.fill_between(x[0:endt], lower, (res_foes[0][0:endt] + res_foes[1][0:endt]), color='b', alpha = 0.1, label = 'Std CO UC')

# naming the x axis
plt.xlabel('time [s]')
# naming the y axis
plt.ylabel('Average CO [mg/s]')

plt.xticks(fontsize=14)
plt.rcParams.update({'font.size': 19})

# giving a title to my graph
plt.title('Average CO emissions over '+str(n_simulations)+' simulations')

# show a legend on the plot
plt.legend(fontsize=18)

# resize the plot  
plt.rcParams["figure.figsize"] = (10, 10)
plt.grid()
plt.xlim([-50,2000])
plt.ylim([-0.4,6])
# function to show the plot
plt.savefig(cars_co_plot +'.pdf')
plt.savefig(cars_co_plot +'.png', transparent=True)

plt.show()

In [ ]:
#PLOT: FUEL agents vs FUEL foes
steps_simulationsA = get_steps(fuel_agents_file, 0)
steps_simulationsF = get_steps(fuel_foes_file, 0)

res_agents, max_stepsA = getMeanSTD2(fuel_agents_file ,steps_simulationsA, 0)
res_foes, max_stepsF = getMeanSTD2(fuel_foes_file , steps_simulationsF, 0)

from scipy.signal import lfilter

n = 80  # the larger n is, the smoother curve will be
b = [1.0 / n] * n
a = 1

res_agents[0][:] = lfilter(b,a,res_agents[0][:])
res_foes[0][:] = lfilter(b,a,res_foes[0][:])

#agents
endt = 40000
x = np.array([t*step_length for t in range(endt)])

        
plt.plot(x[0:endt], res_agents[0][0:endt], label = 'Mean fuel CC', color='r')
lower = res_agents[0][0:endt] - res_agents[1][0:endt]
for xxx in range(len(lower)):
    if lower[xxx] <0:
        lower[xxx] =0
plt.fill_between(x[0:endt], lower, (res_agents[0][0:endt] + res_agents[1][0:endt]), color='r', alpha = 0.1, label = 'Std fuel CC')

#foes
x = np.array([t*step_length for t in range(max_stepsF)])
plt.plot(x[0:endt], res_foes[0][0:endt], label = 'Mean fuel UC', color='b')
lower = res_foes[0][0:endt] - res_foes[1][0:endt]
for xxx in range(len(lower)):
    if lower[xxx] <0:
        lower[xxx] =0
plt.fill_between(x[0:endt], lower, (res_foes[0][0:endt] + res_foes[1][0:endt]), color='b', alpha = 0.1, label = 'Std fuel UC')

# naming the x axis
plt.xlabel('time [s]')
# naming the y axis
plt.ylabel('Average fuel consumption [mg/s]')

plt.xticks(fontsize=14)
plt.rcParams.update({'font.size': 19})

# giving a title to my graph
plt.title('Average fuel consumption over '+str(n_simulations)+' simulations')
# show a legend on the plot
plt.legend(fontsize=18)

# resize the plot  
plt.rcParams["figure.figsize"] = (10, 10)
plt.grid()
plt.xlim([-50,2000])
plt.ylim([-2,80])
# function to show the plot
plt.savefig(cars_fuel_plot +'.pdf')
plt.savefig(cars_fuel_plot +'.png', transparent=True)

plt.show()

In [ ]:
#PLOT: NOISE agents vs NOISE foes
steps_simulationsA = get_steps(noise_agents_file, 0)
steps_simulationsF = get_steps(noise_foes_file, 0)

res_agents, max_stepsA = getMeanSTD2(noise_agents_file ,steps_simulationsA,0)
res_foes, max_stepsF = getMeanSTD2(noise_foes_file , steps_simulationsF, 0)

from scipy.signal import lfilter

n = 30  # the larger n is, the smoother curve will be
b = [1.0 / n] * n
a = 1

res_agents[0][:] = lfilter(b,a,res_agents[0][:])
res_foes[0][:] = lfilter(b,a,res_foes[0][:])

#agents
endt = max_stepsA
x = np.array([t*step_length for t in range(max_stepsA)])
        
plt.plot(x[0:endt], res_agents[0][0:endt], label = 'Mean noise CC', color='r')
lower = res_agents[0][0:endt] - res_agents[1][0:endt]
for xxx in range(len(lower)):
    if lower[xxx] <0:
        lower[xxx] =0
        
plt.fill_between(x[0:endt], lower, (res_agents[0][0:endt] + res_agents[1][0:endt]), color='r', alpha = 0.1, label = 'Std noise CC')

#foes
x = np.array([t*step_length for t in range(max_stepsF)])
    
plt.plot(x[0:endt], res_foes[0][0:endt], label = 'Mean noise UC', color='b')
lower = res_foes[0][0:endt] - res_foes[1][0:endt]
for xxx in range(len(lower)):
    if lower[xxx] <0:
        lower[xxx] =0
plt.fill_between(x[0:endt], lower, (res_foes[0][0:endt] + res_foes[1][0:endt]), color='b', alpha = 0.1, label = 'Std noise UC')

# naming the x axis
plt.xlabel('time [s]')
# naming the y axis
plt.ylabel('Average noise [dB]')

plt.xticks(fontsize=14)
plt.rcParams.update({'font.size': 19})

# giving a title to my graph
plt.title('Average noise emission over '+str(n_simulations)+' simulations')
# show a legend on the plot
plt.legend(fontsize=18)

# resize the plot  
plt.rcParams["figure.figsize"] = (10, 10)
plt.grid()
plt.xlim([-50,2000])
plt.ylim([-0.2,4])
# function to show the plot
plt.savefig(cars_noise_plot +'.pdf')
plt.savefig(cars_noise_plot +'.png', transparent=True)

plt.show()

In [ ]:
# AVERAGE SPEED
arr = np.load(speed_meanagents_file + str(0) + '.npy', allow_pickle=True)
mean_arr = []
for i in range(len(arr)):
    meana = np.mean(arr[i])
    mean_arr.append(meana)
    
print(f"MAX VALUE: {max(mean_arr)}")
print(f"MIN VALUE: {min(mean_arr)}")
print(f"SPEED: {np.mean(mean_arr)} +- {np.std(mean_arr)}")

arr = np.load(speed_meanfoes_file + str(0) + '.npy', allow_pickle=True)
mean_arr = []
for i in range(len(arr)):
    meana = np.mean(arr[i])
    mean_arr.append(meana)
    
print(f"MAX VALUE: {max(mean_arr)}")
print(f"MIN VALUE: {min(mean_arr)}")
print(f"SPEED: {np.mean(mean_arr)} +- {np.std(mean_arr)}")


In [ ]:
#CO2
meanco2a = [0]*n_simulations
meanco2f = [0]*n_simulations

for t in range(n_simulations):
    arr = np.load(co2_meanagents_file + str(t) + '.npy', allow_pickle=True)

    for i in range(len(arr)):
        meanco2a[t] += np.mean(arr[i]) * len(arr[i])*step_length
    
    meanco2a[t] = (meanco2a[t]/len(arr))/1000
mean_arr = []
for i in range(len(meanco2a)):
    if str(meanco2a[i]) != 'nan':
        mean_arr.append(meanco2a[i]) 
    

print(f"mean co2 emitted per agent: {np.mean(mean_arr)} g") 

for t in range(n_simulations):
    arr = np.load(co2_meanfoes_file + str(t) + '.npy', allow_pickle=True)

    for i in range(len(arr)):
        meanco2f[t] += np.mean(arr[i]) * len(arr[i])*step_length
    
    meanco2f[t] = (meanco2f[t]/len(arr))/1000
mean_arr = []
for i in range(len(meanco2f)):
    if str(meanco2f[i]) != 'nan':
        mean_arr.append(meanco2f[i])

print(f"mean co2 emitted per foe: {np.mean(mean_arr)} g") 

# CO

meancoa = [0]*n_simulations
meancof = [0]*n_simulations

for t in range(n_simulations):
    arr = np.load(co_meanagents_file + str(t) + '.npy', allow_pickle=True)

    for i in range(len(arr)):
        meancoa[t] += np.mean(arr[i]) * len(arr[i])*step_length
    
    meancoa[t] = (meancoa[t]/len(arr))/1000
mean_arr = []
for i in range(len(meancoa)):
    if str(meancoa[i]) != 'nan':
        mean_arr.append(meancoa[i])
    
print(f"mean co emitted per agent: {np.mean(mean_arr)} g") 

for t in range(n_simulations):
    arr = np.load(co_meanfoes_file + str(t) + '.npy', allow_pickle=True)

    for i in range(len(arr)):
        meancof[t] += np.mean(arr[i]) * len(arr[i])*step_length
    
    meancof[t] = (meancof[t]/len(arr))/1000
mean_arr = []
for i in range(len(meancof)):
    if str(meancof[i]) != 'nan':
        mean_arr.append(meancof[i])
print(f"mean co emitted per foe: {np.mean(mean_arr)} g") 


# FUEL
meanfuela = [0]*n_simulations
meanfuelf = [0]*n_simulations

for t in range(n_simulations):
    arr = np.load(fuel_meanagents_file + str(t) + '.npy', allow_pickle=True)

    for i in range(len(arr)):
        meanfuela[t] += np.mean(arr[i]) * len(arr[i])*step_length
    
    meanfuela[t] = ((meanfuela[t]/len(arr))/1000)/680
mean_arr = []
for i in range(len(meanfuela)):
    if str(meanfuela[i]) != 'nan':
        mean_arr.append(meanfuela[i])
print(f"mean fuel emitted per agent: {np.mean(mean_arr)} L of gasoline")


for t in range(n_simulations):
    arr = np.load(fuel_meanfoes_file + str(t) + '.npy', allow_pickle=True)

    for i in range(len(arr)):
        meanfuelf[t] += np.mean(arr[i]) * len(arr[i])*step_length
    
    meanfuelf[t] = ((meanfuelf[t]/len(arr))/1000)/680
mean_arr = []
for i in range(len(meanfuelf)):
    if str(meanfuelf[i]) != 'nan':
        mean_arr.append(meanfuelf[i])

print(f"mean fuel emitted per foe: {np.mean(mean_arr)} L of gasoline") 
